In [5]:
from selenium import webdriver  #control the webbrowser and selenium is for automating webbrowser 
from selenium.webdriver.common.by import By   #by is a class which is used to locate the elements on webpage by id,name,class,css,xpath
from selenium.webdriver.common.keys import Keys   #keyboard options
from selenium.webdriver.support.ui import WebDriverWait   #wait for certain condition to occur. because somewebsites are dynamic
from selenium.webdriver.support import expected_conditions as EC   #it is used with webdriverwait to wait until an element is present,clickable,visible
from selenium.webdriver.common.action_chains import ActionChains   #used for chaining multiple actions like moving,clicking,dragging
import time  #used for delays like wait for some seconds to load
import pandas as pd  #for data manipulation and analysis

driver = webdriver.Chrome()    #starting or initializing chrome
wait = WebDriverWait(driver, 30)  # Increased timeout and waiting for particular condition to execute or  be met
#it is used for slow loading pages
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")   #getting into the given link

# Defining a list to store all the route data
rname = []
rlink = []

#defining a function to scrape the data from current page
def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))   #waits until all elements with the class name "route_link" are present on the page. This ensures that the page is fully loaded before attempting to extract data.

    # Loop through each route to extract details
    for route in routescontainer:   #Iterates over each route element found on the page
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text   #extracting the text of the route
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')  #extracting link

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:   #if any error occurs just proceed and continue
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()     #function to extract data from current page
    if page_number < 2:  # Don't try to click next on the last page and prevent clicking next in last page
        try:
            # Locate the pagination container and wait until Pge cont is present
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)    #perform actions like scrolling.
            actions.move_to_element(next_page_button).perform()   #Scrolls the view to ensure the 'Next' button is in view.
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page Waits until the next page is fully loaded and the page number is updated.
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_kerla = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_kerla
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_kerla.csv"
df_kerla.to_csv(path,index=False)    #default df index will be eliminated if we give false


Clicking on page 2
Successfully navigated to page 2


In [7]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_kerla.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(5)

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_kerla = pd.DataFrame(data)
df_bus_kerla

path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_kerla.csv"
df_bus_kerla.to_csv(path,index=False)


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")

# Define a list to store all the route data
rname = []
rlink = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')       

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 3:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_rajasthan = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_rajasthan
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_rajasthan.csv"
df_rajasthan.to_csv(path,index=False)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3


In [20]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_rajasthan.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(5)

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_rajasthan = pd.DataFrame(data)
df_bus_rajasthan

path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_rajasthan.csv"
df_bus_rajasthan.to_csv(path,index=False)


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")

# Define a list to store all the route data
rname = []
rlink = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')       

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_bengal = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_bengal
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bengal.csv"
df_bengal.to_csv(path,index=False)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5


In [14]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_bengal.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(5)

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_bengal = pd.DataFrame(data)
df_bus_bengal

path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_bengal.csv"
df_bus_bengal.to_csv(path,index=False)


In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")

# Define a list to store all the route data
rname = []
rlink = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')       

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_hp = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_hp
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_hp.csv"
df_hp.to_csv(path,index=False)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5


In [16]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_hp.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(5)

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_hp = pd.DataFrame(data)
df_bus_hp

path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_hp.csv"
df_bus_hp.to_csv(path,index=False)


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")

# Define a list to store all the route data
rname = []
rlink = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')       

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_assam = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_assam
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_assam.csv"
df_assam.to_csv(path,index=False)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5


In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_assam.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(5)

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_assam = pd.DataFrame(data)
df_bus_assam

path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_assam.csv"
df_bus_assam.to_csv(path,index=False)


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile")

# Define a list to store all the route data
rname = []
rlink = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')       

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_up = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_up
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_up.csv"
df_up.to_csv(path,index=False)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5


In [6]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_up.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(5)

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_up = pd.DataFrame(data)
df_bus_up

path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_up.csv"
df_bus_up.to_csv(path,index=False)


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")

# Define a list to store all the route data
rname = []
rlink = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')       

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_wb = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_wb
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_wb.csv"
df_wb.to_csv(path,index=False)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5


In [8]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_wb.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(5)

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_wb = pd.DataFrame(data)
df_bus_wb

path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_wb.csv"
df_bus_wb.to_csv(path,index=False)


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")

# Define a list to store all the route data
rname = []
rlink = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')       

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_chand = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_chand
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_chand.csv"
df_chand.to_csv(path,index=False)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5


In [10]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_chand.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(5)

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_chand = pd.DataFrame(data)
df_bus_chand

path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_chand.csv"
df_bus_chand.to_csv(path,index=False)


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")

# Define a list to store all the route data
rname = []
rlink = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')       

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 3:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_punjab = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_punjab
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_punjab.csv"
df_punjab.to_csv(path,index=False)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3


In [12]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_punjab.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(5)

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_punjab = pd.DataFrame(data)
df_bus_punjab

path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_punjab.csv"
df_bus_punjab.to_csv(path,index=False)


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")

# Define a list to store all the route data
rname = []
rlink = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            # Extract route name and link
        
            routename= route.find_element(By.CLASS_NAME, 'route').text
            routelink= route.find_element(By.CLASS_NAME, 'route').get_attribute('href')       

            # Append extracted data to list
            rname.append(routename)
            rlink.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 4:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break


driver.quit()

df_andhra = pd.DataFrame({"routename" : rname,"routelink" : rlink})
df_andhra
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_andhra.csv"
df_andhra.to_csv(path,index=False)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4


In [6]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

driver = webdriver.Chrome()

Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i, j in df_andhra.iterrows():
    link = j["routelink"]
    routes = j["routename"]

    driver.get(link)
    time.sleep(5)
    
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(5)
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
        time.sleep(3)
        
    #scrolling = True
    # while scrolling:
    #     old_page_source = driver.page_source
            
    #         # Use ActionChains to perform a PAGE_DOWN
    #     ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            
    #     time.sleep(5)  
            
    #     new_page_source = driver.page_source
            
    #     if new_page_source == old_page_source:
    #         scrolling = False
    #     time.sleep(3)

    # Scroll to the bottom of the page
    # scroll_height = driver.execute_script("return document.body.scrollHeight")
    # for i in range(0, scroll_height, 100):
    #     driver.execute_script(f"window.scrollTo(0, {i});")
    #     time.sleep(1)
    #     print("Scrolled to bottom")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.CLASS_NAME,'rating-sec')
    except:
        continue
    price = driver.find_elements(By.CSS_SELECTOR,'[class="f-19 f-bold"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

# Ensure all lists have the same length by filling shorter lists with None
max_len = max(len(Bus_names_k), len(Bus_types_k), len(Start_Time_k), len(End_Time_k), len(Total_Duration_k), len(Prices_k), len(Seats_Available_k), len(Ratings_k), len(Route_names), len(Route_links))

def fill_list(lst, max_len):
    while len(lst) < max_len:
        lst.append(None)  # or "" if you prefer empty strings

fill_list(Bus_names_k, max_len)
fill_list(Bus_types_k, max_len)
fill_list(Start_Time_k, max_len)
fill_list(End_Time_k, max_len)
fill_list(Total_Duration_k, max_len)
fill_list(Prices_k, max_len)
fill_list(Seats_Available_k, max_len)
fill_list(Ratings_k, max_len)
fill_list(Route_names, max_len)
fill_list(Route_links, max_len)

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_bus_andhra = pd.DataFrame(data)
df_bus_andhra
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_andhra.csv"
df_bus_andhra.to_csv(path,index=False)

In [ ]:
final=pd.concat([])

In [9]:
df_bus_andhra1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_andhra.csv")

In [10]:
df_bus_assam1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_assam.csv")

In [11]:
df_bus_bengal1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_bengal.csv")

In [12]:
df_bus_chand1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_chand.csv")

In [13]:
df_bus_hp1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_hp.csv")

In [14]:
df_bus_kerla1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_kerla.csv")

In [15]:
df_bus_punjab1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_punjab.csv")

In [16]:
df_bus_up1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_up.csv")

In [17]:
df_bus_wb1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_wb.csv")

In [18]:
df_bus_rajasthan1=pd.read_csv(r"C:/Users/anbar/OneDrive/Desktop/Redbus/df_bus_rajasthan.csv")

In [ ]:
import pandas as pd
complete_busdata=pd.concat([df_bus_kerla1,df_bus_andhra1,df_bus_rajasthan1,df_bus_bengal1,df_bus_hp1,df_bus_assam1,df_bus_up1,df_bus_wb1,df_bus_chand1,df_bus_punjab1],ignore_index=True)

In [ ]:
complete_busdata

In [21]:
complete_busdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10183 entries, 0 to 10182
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Bus_name         10183 non-null  object 
 1   Bus_type         10183 non-null  object 
 2   Start_time       10183 non-null  object 
 3   End_time         10183 non-null  object 
 4   Total_duration   10183 non-null  object 
 5   Price            8352 non-null   float64
 6   Seats_Available  10183 non-null  object 
 7   Ratings          10091 non-null  object 
 8   Route_link       10183 non-null  object 
 9   Route_name       10183 non-null  object 
dtypes: float64(1), object(9)
memory usage: 795.7+ KB


In [22]:
complete_busdata.isnull()  # Returns a DataFrame of the same shape with True where values are NaN



,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
10178,False,False,False,False,False,True,False,False,False,False
10179,False,False,False,False,False,True,False,False,False,False
10180,False,False,False,False,False,True,False,False,False,False
10181,False,False,False,False,False,True,False,False,False,False


In [24]:
complete_busdata.fillna(0)

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,KSRTC (Kerala) - 2063,SWIFT-GARUDA A/C SEATER BUS,22:32,07:35,09h 03m,627.0,2 Seats available,3.6,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,KSRTC (Kerala) - 583,Super Express Non AC Seater Air Bus (2+2),23:30,09:35,10h 05m,469.0,3 Seats available,3.2,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,KSRTC (Kerala) - 56,SWIFT-GARUDA A/C SEATER BUS,23:41,09:05,09h 24m,627.0,1 Seat available,3.5,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,599.0,8 Seats available,4.5,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,MMK Travels,VE A/C Seater / Sleeper (2+1),22:30,07:30,09h 00m,700.0,7 Seats available,4.4,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
10178,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),20:30,05:15,08h 45m,0.0,51 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar
10179,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),15:30,00:15,08h 45m,0.0,51 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar
10180,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),13:00,00:45,11h 45m,0.0,50 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar
10181,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),19:30,07:15,11h 45m,0.0,51 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar


In [ ]:
complete_busdata['Price'].astype('int')

In [27]:
complete_busdata['Price'].fillna(0, inplace=True) 

In [28]:
complete_busdata['Price'] = complete_busdata['Price'].astype('int')

In [29]:
complete_busdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10183 entries, 0 to 10182
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Bus_name         10183 non-null  object
 1   Bus_type         10183 non-null  object
 2   Start_time       10183 non-null  object
 3   End_time         10183 non-null  object
 4   Total_duration   10183 non-null  object
 5   Price            10183 non-null  int64 
 6   Seats_Available  10183 non-null  object
 7   Ratings          10091 non-null  object
 8   Route_link       10183 non-null  object
 9   Route_name       10183 non-null  object
dtypes: int64(1), object(9)
memory usage: 795.7+ KB


In [30]:
complete_busdata["Ratings"]=complete_busdata["Ratings"].str.replace("New","")

In [ ]:
complete_busdata["Ratings"]

In [32]:
complete_busdata["Ratings"].fillna(0,inplace=True)

In [ ]:
complete_busdata

In [39]:
complete_busdata['Ratings'] = pd.to_numeric(complete_busdata['Ratings'], errors='coerce')

In [40]:
complete_busdata['Ratings'] = complete_busdata['Ratings'].astype('float')

In [41]:
complete_busdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10183 entries, 0 to 10182
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Bus_name         10183 non-null  object 
 1   Bus_type         10183 non-null  object 
 2   Start_time       10183 non-null  object 
 3   End_time         10183 non-null  object 
 4   Total_duration   10183 non-null  object 
 5   Price            10183 non-null  int64  
 6   Seats_Available  10183 non-null  object 
 7   Ratings          10072 non-null  float64
 8   Route_link       10183 non-null  object 
 9   Route_name       10183 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 795.7+ KB


In [43]:
import numpy as np

In [44]:
complete_busdata = complete_busdata.replace({np.nan: None})

In [45]:
complete_busdata

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,KSRTC (Kerala) - 2063,SWIFT-GARUDA A/C SEATER BUS,22:32,07:35,09h 03m,627,2 Seats available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,KSRTC (Kerala) - 583,Super Express Non AC Seater Air Bus (2+2),23:30,09:35,10h 05m,469,3 Seats available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,KSRTC (Kerala) - 56,SWIFT-GARUDA A/C SEATER BUS,23:41,09:05,09h 24m,627,1 Seat available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,599,8 Seats available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,MMK Travels,VE A/C Seater / Sleeper (2+1),22:30,07:30,09h 00m,700,7 Seats available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
10178,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),20:30,05:15,08h 45m,0,51 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar
10179,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),15:30,00:15,08h 45m,0,51 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar
10180,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),13:00,00:45,11h 45m,0,50 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar
10181,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),19:30,07:15,11h 45m,0,51 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar


In [46]:
path=r"C:/Users/anbar/OneDrive/Desktop/Redbus/complete_busdata.csv"
complete_busdata.to_csv(path,index=False)

In [1]:
import pandas as pd
data=pd.read_csv("C:/Users/anbar/OneDrive/Desktop/Redbus/complete_busdata.csv")

In [26]:
data.fillna(0)

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,KSRTC (Kerala) - 2063,SWIFT-GARUDA A/C SEATER BUS,22:32,07:35,09h 03m,627,2 Seats available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,KSRTC (Kerala) - 583,Super Express Non AC Seater Air Bus (2+2),23:30,09:35,10h 05m,469,3 Seats available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,KSRTC (Kerala) - 56,SWIFT-GARUDA A/C SEATER BUS,23:41,09:05,09h 24m,627,1 Seat available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,Silver Line Travels,NON A/C Semi Sleeper (2+2),22:00,06:10,08h 10m,599,8 Seats available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,MMK Travels,VE A/C Seater / Sleeper (2+1),22:30,07:30,09h 00m,700,7 Seats available,0.0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
10178,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),20:30,05:15,08h 45m,0,51 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar
10179,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),15:30,00:15,08h 45m,0,51 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar
10180,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),13:00,00:45,11h 45m,0,50 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar
10181,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),19:30,07:15,11h 45m,0,51 Seats available,4.5,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Jalandhar


In [ ]:
!pip install mysql-connector-python

In [17]:
#connecting with database
import mysql.connector
mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="Sql@10",


)


print(mydb)
mycursor = mydb.cursor(buffered=True)

In [18]:
mycursor.execute("create database redbus")

In [19]:
mycursor.execute("use redbus")

In [20]:
mycursor.execute("create table busdetails (id INT AUTO_INCREMENT PRIMARY KEY,bus_name VARCHAR(150), bus_type VARCHAR(150), start_time VARCHAR(50), end_time VARCHAR(50), Total_duration VARCHAR(50), price INT, Seats_available VARCHAR(100), Ratings FLOAT, route_link VARCHAR(200), route_name VARCHAR(100))")


In [21]:
insert_query = '''
INSERT INTO busdetails(Bus_name, Bus_type, Start_time, End_time, Total_duration, Price, Seats_Available, Ratings, Route_link, Route_name)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''


In [22]:
import pandas as pd
data=pd.read_csv("C:/Users/anbar/OneDrive/Desktop/Redbus/complete_busdata.csv")

In [23]:
data=data.fillna(0)

In [ ]:
data

In [24]:
data_final1 = data.values.tolist()

In [25]:
data_final1
a= [tuple(i) for i in data_final1]


In [26]:

mycursor.executemany(insert_query,a)
mydb.commit()

In [ ]:
# for i,row in enumerate(a):
#     print(i)
#     mycursor.execute(insert_query, row)
#     mydb.commit()

In [14]:
# mycursor.execute("SELECT * FROM busdetails")

# myresult = mycursor.fetchall()

In [ ]:
# pd.DataFrame(myresult)

In [16]:
# mycursor.execute('drop database redbus')